In [3]:
import ISS_decoding.SpaceTx_format as STX
import ISS_decoding.decoding as DEC
import ISS_decoding.qc_metrics as QC
import pandas as pd

ModuleNotFoundError: No module named 'starfish'

# Specify the path where your sections are saved

In [ ]:
sample = '/media/external/eae_processing/batch_3/R3_B_BOTH_PEAK/'


# Format the data in SpaceTx format

The first thing that we need to do before we can start the actual decoding is to conform our images (the resliced tiles) to SpaceTx compatible data, also known as the SpaceTx format. To read more about the SpaceTx format, read the following: GitHub - spacetx/sptx-format: Image data format specification for spaceTx. 


In [5]:
STX.make_spacetx_format(path = sample,
codebook_csv = '/home/cml/Downloads/codebook_no_Plp1.csv',
filenames=['Base_1_stitched', 'Base_2_stitched', 
           'Base_3_stitched', 'Base_4_stitched', 
           'Base_5_stitched','Base_6_stitched'],
tile_dim=2000,
pixelscale=0.1625,
channels=["AF750", "Cy5", "Cy3", "AF488", "DAPI"],
DO_decorators=["AF750", "Cy5", "Cy3", "AF488"],
folder_spacetx='SpaceTX_format',
nuclei_channel=5,
)

NameError: name 'STX' is not defined

The process_experiment  function is run in order to decode the ISS experiment. The function will take every individual tile formatted in the SpaceTx format, detect every spot and assign to it the most probable code. Finally, the function will extract the XY position and identity of every spot that has been decoded in the tile

In [ ]:
DEC.process_experiment(exp_path = sample+'/SpaceTX_format/experiment.json', 
output = sample+'/starfish_output_quality_scores_spots_alignment_new/', 
register = True,
register_dapi = False,
masking_radius = 15, 
threshold = 0.002, 
sigma_vals = [1, 10, 30], # min, max and number
decode_mode = 'PRMC',
normalization_method = 'MH' # or MD
                      )

# Put all the fovs together

As we mentioned previously, every single tile is decoded independently. For this reason, the last step when decoding is to concatenate all the csv’s from individual tiles . For this, we use the following function.


In [ ]:
concatenate_starfish_output(path=sample+'/starfish_output_quality_scores_spots_alignment_new/',
                            outpath=sample+'/starfish_output_quality_scores_spots_alignment_new/'tag='decoding1')

# Read csv's

After decoding, we read the csv containing the spots

In [ ]:
reads=pd.read_csv('')

# Filter reads

In [ ]:
QC.reads_filt=filter_reads(reads,min_quality_mean=0.2)

# Plot expression

In [ ]:
Plot the gene expression decoded of your tissue. Parameters xcolumn and y column define the XY positions of your spots. Key is the column containing the gene identity of your spots. By changing 'genes'

In [ ]:
QC.plot_expression(reads,key='target',colorcode=['red'],xcolumn='xc',ycolumn='yc',genes='individual',size=8,background='black',title_color='white',figuresize=(10,10),save=None,format='pdf')

# Quality metrics

There are several functions that allow you to explore how good your data is. In this section we describe the main functionalities that can be used to explore the quality of our dataset:


In [ ]:
QC.quality_per_cycle(reads,cycles=6)

quality_per_gene: this function is designed to evaluate the quality of the reads assigned to each gene. Since every gene has an associated barcode with a specific sequence of colors across cycles, we might have a big quality bias between different genes just because of the barcode composition

In [ ]:
QC.quality_per_gene(reads,on='qcscore_mean',gene_name='target')

quality_per_base: represents as a violin plot the qualities obtained for the decoded spots on each base

In [ ]:
QC.plot_frequencies(reads,on='target')

Compare_scores: this function is used to plot more than one quality score at the same time. It represents the same as plot_scores, but for 2 quality metrics. It can facilitate the interpretation of how some metrics change

In [ ]:
QC.compare_scores(reads,score1='quality_mean',score2='quality_minimum',hue='assigned',kind='hist',color='#3266a8')

Plot_scores: allows us to plot in the form of a histogram any quality parameter that we might have in our dataset. If decoding using “PRMC”, where NaNs are detected, we can also plot the number of NaNs (non assigned) vs TRUE (assigned) reads against one of our quality scores. 

In [ ]:
QC.plot_scores(reads,on='quality_mean',hue='assigned',log_scale=False) 